In [1]:
# Import the dependency
import pandas as pd
from sqlalchemy import create_engine

# Step 1: Extract


In [3]:
# Load the data into a Pandas DataFrame
file_path = 'Data/all_data_M_2023.xlsx'
all_owes_df = pd.read_excel(file_path) #, sheet_name='All May 2023 data')

# Step 2: Transform the data

In [12]:
# Narrow down the data to State level
state_oews_df = all_owes_df.loc[all_owes_df['AREA_TYPE'] == 2, :]
state_oews_df.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
177501,1,Alabama,2,AL,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,19.88,30.09,46.18,22620,29580,41350,62580,96050,NaN,NaN
177502,2,Alaska,2,AK,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,26.99,40.52,58.35,31200,38720,56140,84280,121370,NaN,NaN
177503,4,Arizona,2,AZ,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,22.92,35.05,51.67,30870,36150,47680,72900,107470,NaN,NaN
177504,5,Arkansas,2,AR,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,18.78,28.32,40.5,26360,30000,39060,58900,84240,NaN,NaN
177505,6,California,2,CA,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,25.98,44.83,73.07,34170,37890,54030,93250,151990,NaN,NaN


In [13]:
# Select the columns we need
columns_to_keep = [
    'AREA', 
    'AREA_TITLE', 
    'PRIM_STATE',
    'OCC_CODE', 
    'OCC_TITLE', 
    'O_GROUP',
    'TOT_EMP',
    'EMP_PRSE',
    'JOBS_1000',
    'LOC_QUOTIENT',
    'H_MEAN',
    'A_MEAN',
    'MEAN_PRSE',
    'H_PCT25',
    'H_MEDIAN',
    'H_PCT75',
    'A_PCT25',
    'A_MEDIAN',
    'A_PCT75'
]

cleaned_df = state_oews_df[columns_to_keep]
cleaned_df.head()

,AREA,AREA_TITLE,PRIM_STATE,OCC_CODE,OCC_TITLE,O_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC_QUOTIENT,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT25,H_MEDIAN,H_PCT75,A_PCT25,A_MEDIAN,A_PCT75
177501,1,Alabama,AL,00-0000,All Occupations,total,2053090,0,1000,1,25.67,53400,0.2,14.22,19.88,30.09,29580,41350,62580
177502,2,Alaska,AK,00-0000,All Occupations,total,311610,0,1000,1,33.6,69880,0.3,18.62,26.99,40.52,38720,56140,84280
177503,4,Arizona,AZ,00-0000,All Occupations,total,3129720,0,1000,1,30.31,63040,0.5,17.38,22.92,35.05,36150,47680,72900
177504,5,Arkansas,AR,00-0000,All Occupations,total,1271320,0,1000,1,24.64,51250,0.2,14.42,18.78,28.32,30000,39060,58900
177505,6,California,CA,00-0000,All Occupations,total,17945910,0,1000,1,37,76960,0.4,18.21,25.98,44.83,37890,54030,93250


In [14]:
# Rename the columns for preparing to load data into database
renamed_oews_df = cleaned_df.rename(columns={
    'AREA': 'area',
    'AREA_TITLE': 'area_title',
    'PRIM_STATE': 'prim_state',
    'OCC_CODE': 'occ_code',
    'OCC_TITLE': 'occ_title',
    'O_GROUP': 'o_group',
    'TOT_EMP': 'tot_emp',
    'EMP_PRSE': 'emp_prse',
    'JOBS_1000': 'jobs_1000',
    'LOC_QUOTIENT': 'loc_quotient',
    'H_MEAN': 'h_mean',
    'A_MEAN': 'a_mean',
    'MEAN_PRSE': 'mean_prse',
    'H_PCT25': 'h_pct25',
    'H_MEDIAN': 'h_median',
    'H_PCT75': 'h_pct75',
    'A_PCT25': 'a_pct25',
    'A_MEDIAN': 'a_median',
    'A_PCT75': 'a_pct75'
})
renamed_oews_df.head()

,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct25,h_median,h_pct75,a_pct25,a_median,a_pct75
177501,1,Alabama,AL,00-0000,All Occupations,total,2053090,0,1000,1,25.67,53400,0.2,14.22,19.88,30.09,29580,41350,62580
177502,2,Alaska,AK,00-0000,All Occupations,total,311610,0,1000,1,33.6,69880,0.3,18.62,26.99,40.52,38720,56140,84280
177503,4,Arizona,AZ,00-0000,All Occupations,total,3129720,0,1000,1,30.31,63040,0.5,17.38,22.92,35.05,36150,47680,72900
177504,5,Arkansas,AR,00-0000,All Occupations,total,1271320,0,1000,1,24.64,51250,0.2,14.42,18.78,28.32,30000,39060,58900
177505,6,California,CA,00-0000,All Occupations,total,17945910,0,1000,1,37,76960,0.4,18.21,25.98,44.83,37890,54030,93250


In [15]:
# Select specific occupation codes to narrow down the dataset
occ_code_list = ['00-0000', '11-0000', '13-0000', '15-0000', '17-0000', '19-0000', '21-0000', '23-0000', '25-0000', '27-0000', '29-0000', '31-0000', '33-0000',\
    '35-0000', '37-0000', '39-0000', '41-0000', '43-0000', '45-0000', '47-0000', '49-0000', '51-0000', '53-0000', '11-3021', '13-1161', '13-2031', '13-2041',\
        '13-2054', '15-1221', '15-1251', '15-1252', '15-1253', '15-1254', '15-2011', '15-2031', '15-2041', '19-1022', '19-1029', '19-2041', '29-9091', '41-3021',\
            '43-9021', '13-2051','15-1211', '15-1242', '15-1243', '15-2051']

filtered_df = renamed_oews_df[renamed_oews_df['occ_code'].isin(occ_code_list)]
filtered_df

,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct25,h_median,h_pct75,a_pct25,a_median,a_pct75
177501,1,Alabama,AL,00-0000,All Occupations,total,2053090,0,1000,1,25.67,53400,0.2,14.22,19.88,30.09,29580,41350,62580
177502,2,Alaska,AK,00-0000,All Occupations,total,311610,0,1000,1,33.6,69880,0.3,18.62,26.99,40.52,38720,56140,84280
177503,4,Arizona,AZ,00-0000,All Occupations,total,3129720,0,1000,1,30.31,63040,0.5,17.38,22.92,35.05,36150,47680,72900
177504,5,Arkansas,AR,00-0000,All Occupations,total,1271320,0,1000,1,24.64,51250,0.2,14.42,18.78,28.32,30000,39060,58900
177505,6,California,CA,00-0000,All Occupations,total,17945910,0,1000,1,37,76960,0.4,18.21,25.98,44.83,37890,54030,93250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212239,51,Virginia,VA,53-0000,Transportation and Material Moving Occupations,major,309810,0.8,77.82,0.86,22.71,47240,0.4,16.21,19.52,24.61,33710,40600,51190
212240,53,Washington,WA,53-0000,Transportation and Material Moving Occupations,major,285510,0.8,81.678,0.9,27.17,56510,0.4,18.6,22.67,29.22,38690,47150,60770
212241,54,West Virginia,WV,53-0000,Transportation and Material Moving Occupations,major,56980,1.3,82.745,0.91,19.14,39810,0.6,13.97,17.07,22.06,29050,35500,45890
212242,55,Wisconsin,WI,53-0000,Transportation and Material Moving Occupations,major,276680,0.6,95.87,1.06,21.51,44730,0.4,16.46,20.28,24.99,34240,42170,51980


In [17]:
# Check the data type before loading them into database
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2361 entries, 177501 to 212243
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   area          2361 non-null   int64 
 1   area_title    2361 non-null   object
 2   prim_state    2361 non-null   object
 3   occ_code      2361 non-null   object
 4   occ_title     2361 non-null   object
 5   o_group       2361 non-null   object
 6   tot_emp       2361 non-null   object
 7   emp_prse      2361 non-null   object
 8   jobs_1000     2361 non-null   object
 9   loc_quotient  2361 non-null   object
 10  h_mean        2361 non-null   object
 11  a_mean        2361 non-null   object
 12  mean_prse     2361 non-null   object
 13  h_pct25       2361 non-null   object
 14  h_median      2361 non-null   object
 15  h_pct75       2361 non-null   object
 16  a_pct25       2361 non-null   object
 17  a_median      2361 non-null   object
 18  a_pct75       2361 non-null   object
dtypes: i

In [18]:
# The list of columns to convert the data type
columns_to_convert = [
    'tot_emp',
    'emp_prse',
    'jobs_1000',
    'loc_quotient',
    'h_mean',
    'a_mean',
    'mean_prse',
    'h_pct25',
    'h_median',
    'h_pct75',
    'a_pct25',
    'a_median',
    'a_pct75'
]

In [19]:
# Convert the specified columns to numeric, coercing errors
filtered_df[columns_to_convert] = filtered_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2361 entries, 177501 to 212243
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          2361 non-null   int64  
 1   area_title    2361 non-null   object 
 2   prim_state    2361 non-null   object 
 3   occ_code      2361 non-null   object 
 4   occ_title     2361 non-null   object 
 5   o_group       2361 non-null   object 
 6   tot_emp       2343 non-null   float64
 7   emp_prse      2343 non-null   float64
 8   jobs_1000     2343 non-null   float64
 9   loc_quotient  2343 non-null   float64
 10  h_mean        2300 non-null   float64
 11  a_mean        2351 non-null   float64
 12  mean_prse     2351 non-null   float64
 13  h_pct25       2300 non-null   float64
 14  h_median      2300 non-null   float64
 15  h_pct75       2296 non-null   float64
 16  a_pct25       2351 non-null   float64
 17  a_median      2351 non-null   float64
 18  a_pct75       2347 non-nul

/var/folders/16/qt_t7c5n31q5g2k_npqy3ryw0000gn/T/ipykernel_70518/3587210395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[columns_to_convert] = filtered_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')


In [20]:
# Drop the data that are not available
final_df = filtered_df.dropna(how='any')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2278 entries, 177501 to 212243
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          2278 non-null   int64  
 1   area_title    2278 non-null   object 
 2   prim_state    2278 non-null   object 
 3   occ_code      2278 non-null   object 
 4   occ_title     2278 non-null   object 
 5   o_group       2278 non-null   object 
 6   tot_emp       2278 non-null   float64
 7   emp_prse      2278 non-null   float64
 8   jobs_1000     2278 non-null   float64
 9   loc_quotient  2278 non-null   float64
 10  h_mean        2278 non-null   float64
 11  a_mean        2278 non-null   float64
 12  mean_prse     2278 non-null   float64
 13  h_pct25       2278 non-null   float64
 14  h_median      2278 non-null   float64
 15  h_pct75       2278 non-null   float64
 16  a_pct25       2278 non-null   float64
 17  a_median      2278 non-null   float64
 18  a_pct75       2278 non-nul

# Step 3: Load the Data

In [34]:
# Export the data to CSV
final_df.to_csv('data/EmploymentWageData.csv', index=False)